In [1]:
#from keras.datasets import mnist
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from google.colab import drive
import torch
from keras.utils import np_utils
size_image = [28, 56, 100]
drive.mount('/content/drive')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.mnist.load_data()
print(X_train.shape)
print(Y_train.shape)

Mounted at /content/drive
11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)


In [2]:
label = len(set(list(Y_train)))
Y_test = torch.from_numpy(Y_test)
Y_test = torch.tensor(Y_test, dtype = torch.long).to(device)
Y_train = torch.from_numpy(Y_train)
Y_train = torch.tensor(Y_train, dtype = torch.long).to(device)
print("Y_train shape is : {}".format(Y_train.shape))
print("Y_test shape is : {}".format(Y_test.shape))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


Y_train shape is : torch.Size([60000])
Y_test shape is : torch.Size([10000])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [3]:
from torch import nn
from torch.nn import functional
from torch.optim import Adam
from torch.optim import SGD
from torch.nn import Softmax
class Model(nn.Module):
  def __init__(self, row, col):
    super().__init__()
    self.row = row
    self.col = col
    self.conv1 = nn.Conv2d(1, 32, 3, padding = 1)
    self.conv2 = nn.Conv2d(32, 32, 3, padding = 1)
    self.linear1 = nn.Linear(int(row / 4 ) * int(col / 4) * 32, 128)
    self.linear2 = nn.Linear(128, 128)
    self.linear3 = nn.Linear(128, label)
    self.pool = nn.MaxPool2d(2, 2)
  def forward(self, data):
    data = self.pool(functional.relu(self.conv1(data)))
    data = self.pool(functional.relu(self.conv2(data)))
    data = torch.flatten(data, 1)
    data = functional.relu(self.linear1(data))
    data = functional.relu(self.linear2(data))
    data = self.linear3(data)
    #data = self.softmax(data)
    return data
class Model2(nn.Module):
  def __init__(self, row, col):
    super().__init__()
    self.row = row
    self.col = col
    self.conv1 = nn.Conv2d(1, 32, 3, padding = 1)
    self.linear1 = nn.Linear(int(row / 2 ) * int(col / 2) * 32, 128)
    self.linear2 = nn.Linear(128, 128)
    self.linear3 = nn.Linear(128, label)
    self.pool = nn.MaxPool2d(2, 2)
  def forward(self, data):
    data = self.pool(functional.relu(self.conv1(data)))
    data = torch.flatten(data, 1)
    data = functional.relu(self.linear1(data))
    data = functional.relu(self.linear2(data))
    data = self.linear3(data)
    #data = self.softmax(data)
    return data
softmax = Softmax(dim = 1)
model_arr = []
optimizer_arr = []
for size in size_image:
  model = Model(size, size)
  model = model.to(device)
  model = model.double()
  model_arr.append(model)
  model = Model2(size, size)
  model = model.to(device)
  model = model.double()
  model_arr.append(model)
for i in range(len(model_arr)):
  optimizer_arr.append(Adam(model_arr[i].parameters(), lr = 0.00002))
Cross_entrophy = nn.CrossEntropyLoss()
#optimizer = SGD(model.parameters(), lr = 0.00001, momentum = 0.9)
epochs_number = 12
batch_size = 32
for epoch in range(epochs_number):
   for i in range(len(model_arr)):
     split = int(X_train.shape[0] / batch_size)
     random = np.random.permutation(split)
     loss_sum = 0
     for j in random:
      Xj = X_train[j * batch_size : min(j * batch_size + batch_size, X_train.shape[0])]
      Yj = Y_train[j * batch_size : min(j * batch_size + batch_size, X_train.shape[0])]
      Xj = np.expand_dims(Xj, axis=-1)
      Xj = tf.image.resize(Xj, [model_arr[i].row, model_arr[i].col])
      Xj = np.array(Xj).reshape(Xj.shape[0], 1, Xj.shape[1] , Xj.shape[2]).astype(np.float32)/255.
      Xj = torch.from_numpy(Xj)
      Xj = torch.tensor(Xj, dtype = torch.double).to(device)
      output = model_arr[i].forward(Xj)
      loss = Cross_entrophy(output, Yj)
      loss.backward()
      optimizer_arr[i].step()
      loss_sum += loss.item()
      del loss
      del output
     if epoch % 3 == 0:
       print("Loss value of model {} is : {}".format(i, loss_sum))
   if epoch % 3 == 0:
     print(" ")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Loss value of model 0 is : 1799.9515801204977
Loss value of model 1 is : 1833.4544816340524
Loss value of model 2 is : 1471.5895875867172
Loss value of model 3 is : 1743.354845239232
Loss value of model 4 is : 1228.0720250703307
Loss value of model 5 is : 1769.0770613708728
 
Loss value of model 0 is : 175.3290947886888
Loss value of model 1 is : 355.12527933758844
Loss value of model 2 is : 134.35932979816863
Loss value of model 3 is : 406.86000575659165
Loss value of model 4 is : 209.17089725742156
Loss value of model 5 is : 531.9913612415816
 
Loss value of model 0 is : 111.39843725747554
Loss value of model 1 is : 194.30627293452497
Loss value of model 2 is : 97.44182440652695
Loss value of model 3 is : 216.83045791866917
Loss value of model 4 is : 128.27309793509897
Loss value of model 5 is : 266.9683225050497
 
Loss value of model 0 is : 77.45658066706596
Loss value of model 1 is : 121.31248356106242
Loss value of model 2 is : 75.56627731106204
Loss value of model 3 is : 121.9842

In [8]:
def score(X, Y, model, train_test, softmax):
  multi_false = 0
  false = [0 for i in range(len(model_arr))]
  for i in range(X.shape[0]):
    output = None
    for j in range(len(model_arr)):
      Xj = np.expand_dims(X[i], axis=-1)
      #Xj = Xj.reshape(1, Xj.shape[0], Xj.shape[1])
      Xj = tf.image.resize(Xj, [model_arr[j].row, model_arr[j].col])
      Xj = np.array(Xj)
      Xj = torch.from_numpy(Xj)
      Xj = torch.tensor(Xj, dtype = torch.double).to(device)
      Xj = torch.reshape(Xj, (1, 1, model_arr[j].row, model_arr[j].col))
      output_j = model_arr[j].forward(Xj)
      output_j = softmax(output_j)
      predict_j = torch.argmax(output_j[0])
      if predict_j != Y[i]:
        false[j] += 1
      if j == 0:
        output = output_j
      else:
        output += output_j
    output /= float(len(model_arr))
    predict = torch.argmax(output[0])
    if predict != Y[i]:
      multi_false += 1
  for i in range(len(model_arr)):
    print("DNN model {} {} score is {}%".format(i, train_test, (100 * (X.shape[0] - false[i]) / X.shape[0])))
  print("Multi DNN model {} score is {}%".format(train_test, (100 * (X.shape[0] -  multi_false) / X.shape[0])))
score(X_train, Y_train, model, "Train", softmax)
score(X_test, Y_test, model, "Test", softmax)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


DNN model 0 Train score is 77.69%
DNN model 1 Train score is 89.56833333333333%
DNN model 2 Train score is 89.66333333333333%
DNN model 3 Train score is 96.54166666666667%
DNN model 4 Train score is 95.86833333333334%
DNN model 5 Train score is 89.72%
Multi DNN model Train score is 97.6%
DNN model 0 Test score is 78.16%
DNN model 1 Test score is 89.12%
DNN model 2 Test score is 88.96%
DNN model 3 Test score is 95.67%
DNN model 4 Test score is 94.55%
DNN model 5 Test score is 89.67%
Multi DNN model Test score is 96.4%
